# Blood Cells Classification

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint,CSVLogger
import tensorflow as tf 
from keras import backend as k
from keras.models import load_model
from PIL import Image
import os
from sklearn.utils import shuffle
import scipy.io as sio
from skimage import io
from random import shuffle
from scipy import misc
from tqdm import tqdm
import matplotlib.pyplot as plt

Using TensorFlow backend.


## Data Preparation:

In [2]:
def get_testing_data_2d():
    
    EOSINOPHIL = os.listdir("TEST/EOSINOPHIL/")
    LYMPHOCYTE = os.listdir("TEST/LYMPHOCYTE/")
    MONOCYTE   = os.listdir("TEST/MONOCYTE/")
    NEUTROPHIL = os.listdir("TEST/NEUTROPHIL/")
    

    testing_data = []
    

    for (imgE, imgL, imgM, imgN) in tqdm(zip(EOSINOPHIL, LYMPHOCYTE, MONOCYTE, NEUTROPHIL)):

        img_e = io.imread('TEST/EOSINOPHIL/' + imgE , as_grey=True)
        img_l = io.imread('TEST/LYMPHOCYTE/' + imgL , as_grey=True)
        img_m = io.imread('TEST/MONOCYTE/'   + imgM , as_grey=True)
        img_n = io.imread('TEST/NEUTROPHIL/' + imgN , as_grey=True)
        
        img_e = misc.imresize(img_e , (240, 240))
        img_l = misc.imresize(img_l , (240, 240))
        img_m = misc.imresize(img_m , (240, 240))
        img_n = misc.imresize(img_n , (240, 240))
        
        testing_data.append([np.array(img_e), 1])
        testing_data.append([np.array(img_l), 2])
        testing_data.append([np.array(img_m), 3])
        testing_data.append([np.array(img_n), 4])
        
    #imgs = np.array(imgs)

    #images = imgs[:,0]
    #labels = imgs[:,1]
            
    #return images,labels
    
    shuffle(testing_data)
    
    return testing_data

def get_training_data_2d():
    
    EOSINOPHIL = os.listdir("TRAIN/EOSINOPHIL/")
    LYMPHOCYTE = os.listdir("TRAIN/LYMPHOCYTE/")
    MONOCYTE   = os.listdir("TRAIN/MONOCYTE/")
    NEUTROPHIL = os.listdir("TRAIN/NEUTROPHIL/")
    

    training_data = []
    

    for (imgE, imgL, imgM, imgN) in tqdm(zip(EOSINOPHIL, LYMPHOCYTE, MONOCYTE, NEUTROPHIL)):

        img_e = io.imread('TRAIN/EOSINOPHIL/' + imgE , as_grey=True)
        img_l = io.imread('TRAIN/LYMPHOCYTE/' + imgL , as_grey=True)
        img_m = io.imread('TRAIN/MONOCYTE/'   + imgM , as_grey=True)
        img_n = io.imread('TRAIN/NEUTROPHIL/' + imgN , as_grey=True)
        
        img_e = misc.imresize(img_e , (240, 240))
        img_l = misc.imresize(img_l , (240, 240))
        img_m = misc.imresize(img_m , (240, 240))
        img_n = misc.imresize(img_n , (240, 240))
        
        training_data.append([np.array(img_e), 1])
        training_data.append([np.array(img_l), 2])
        training_data.append([np.array(img_m), 3])
        training_data.append([np.array(img_n), 4])
        
    #imgs = np.array(imgs)

    #images = imgs[:,0]
    #labels = imgs[:,1]
            
    #return images,labels
    
    shuffle(training_data)
    
    return training_data

In [3]:
training_data = get_training_data_2d()
testing_data  = get_testing_data_2d()

0it [00:00, ?it/s]C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
2478it [04:48,  8.59it/s]
0it [00:00, ?it/s]C:\Users\DELL\Anaconda3\lib\si

In [4]:
x_train = np.array([i[0] for i in training_data]).reshape(-1,240,240,1)
y_train = [i[1] for i in training_data]

x_test = np.array([i[0] for i in testing_data]).reshape(-1,240,240,1)
y_test = [i[1] for i in testing_data]

In [5]:
y_train[0]

3

In [6]:
batch_size = 100
nr_classes = 5
nr_iterations = 5
saved_weights_name='SVMWeights.h5'

x_train = x_train.astype('float32')

x_test = x_test.astype('float32')

x_train /= 255

x_test /= 255

y_train = np_utils.to_categorical(y_train, nr_classes)

y_test = np_utils.to_categorical(y_test, nr_classes)

In [7]:
y_train = y_train[:,1:5]
y_test = y_test[:,1:5]

## Creating model using Keras:

In [8]:
input_shape=(240,240,1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(5, 5)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(5, 5)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='relu'))

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])

saved_weights_name='CNNWeights.h5'

checkpoint = ModelCheckpoint(saved_weights_name, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('v.csv')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 236, 236, 32)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 43, 43, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               524416    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
Total para

In [9]:
history = model.fit(x_train, y_train, batch_size = batch_size, nb_epoch = nr_iterations, shuffle = True, verbose = 1, validation_data = (x_test, y_test) ,callbacks=[checkpoint,csv_logger])

C:\Users\DELL\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 9912 samples, validate on 2480 samples
Epoch 1/5
9912/9912 [==============================] - ETA: 13:35 - loss: 5.8922 - acc: 0.25 - ETA: 12:14 - loss: 3.8559 - acc: 0.27 - ETA: 11:46 - loss: 3.0573 - acc: 0.27 - ETA: 11:21 - loss: 2.6673 - acc: 0.26 - ETA: 11:19 - loss: 2.4134 - acc: 0.26 - ETA: 11:15 - loss: 2.2443 - acc: 0.26 - ETA: 11:12 - loss: 2.1230 - acc: 0.25 - ETA: 11:12 - loss: 2.0306 - acc: 0.25 - ETA: 11:08 - loss: 1.9596 - acc: 0.25 - ETA: 11:05 - loss: 1.9014 - acc: 0.25 - ETA: 11:00 - loss: 1.8536 - acc: 0.25 - ETA: 10:55 - loss: 1.8134 - acc: 0.26 - ETA: 10:52 - loss: 1.7850 - acc: 0.26 - ETA: 10:48 - loss: 1.7571 - acc: 0.26 - ETA: 10:46 - loss: 1.7322 - acc: 0.26 - ETA: 10:44 - loss: 1.7101 - acc: 0.26 - ETA: 10:37 - loss: 1.6907 - acc: 0.26 - ETA: 10:29 - loss: 1.6738 - acc: 0.27 - ETA: 10:20 - loss: 1.6584 - acc: 0.27 - ETA: 10:10 - loss: 1.6444 - acc: 0.27 - ETA: 10:02 - loss: 1.6322 - acc: 0.27 - ETA: 9:55 - loss: 1.6213 - acc: 0.2709 - ETA: 9:47 - loss

9912/9912 [==============================] - ETA: 12:37 - loss: 1.3835 - acc: 0.26 - ETA: 12:23 - loss: 1.3976 - acc: 0.24 - ETA: 12:28 - loss: 1.3945 - acc: 0.24 - ETA: 12:16 - loss: 1.3941 - acc: 0.23 - ETA: 12:07 - loss: 1.3933 - acc: 0.24 - ETA: 12:07 - loss: 1.3899 - acc: 0.26 - ETA: 11:58 - loss: 1.3921 - acc: 0.25 - ETA: 11:48 - loss: 1.3939 - acc: 0.24 - ETA: 11:38 - loss: 1.3927 - acc: 0.24 - ETA: 11:28 - loss: 1.3931 - acc: 0.24 - ETA: 11:20 - loss: 1.3927 - acc: 0.24 - ETA: 11:11 - loss: 1.3923 - acc: 0.24 - ETA: 11:03 - loss: 1.3905 - acc: 0.25 - ETA: 10:55 - loss: 1.3904 - acc: 0.25 - ETA: 10:48 - loss: 1.3915 - acc: 0.25 - ETA: 10:41 - loss: 1.3910 - acc: 0.25 - ETA: 10:34 - loss: 1.3907 - acc: 0.25 - ETA: 10:30 - loss: 1.3902 - acc: 0.25 - ETA: 10:23 - loss: 1.3906 - acc: 0.25 - ETA: 10:15 - loss: 1.3907 - acc: 0.25 - ETA: 10:08 - loss: 1.3901 - acc: 0.25 - ETA: 10:01 - loss: 1.3900 - acc: 0.25 - ETA: 9:53 - loss: 1.3901 - acc: 0.2552 - ETA: 9:45 - loss: 1.3901 - acc: 0.


Epoch 00003: val_acc improved from 0.25000 to 0.25000, saving model to CNNWeights.h5
Epoch 4/5
9912/9912 [==============================] - ETA: 14:09 - loss: 1.4156 - acc: 0.20 - ETA: 13:46 - loss: 1.4060 - acc: 0.20 - ETA: 13:41 - loss: 1.3976 - acc: 0.25 - ETA: 13:39 - loss: 1.3970 - acc: 0.25 - ETA: 13:29 - loss: 1.3944 - acc: 0.25 - ETA: 13:11 - loss: 1.3917 - acc: 0.26 - ETA: 12:56 - loss: 1.3921 - acc: 0.26 - ETA: 12:42 - loss: 1.3926 - acc: 0.25 - ETA: 12:31 - loss: 1.3914 - acc: 0.26 - ETA: 12:20 - loss: 1.3905 - acc: 0.26 - ETA: 12:10 - loss: 1.3900 - acc: 0.26 - ETA: 12:01 - loss: 1.3888 - acc: 0.26 - ETA: 11:51 - loss: 1.3891 - acc: 0.26 - ETA: 11:40 - loss: 1.3887 - acc: 0.26 - ETA: 11:31 - loss: 1.3884 - acc: 0.26 - ETA: 11:21 - loss: 1.3885 - acc: 0.26 - ETA: 11:11 - loss: 1.3887 - acc: 0.26 - ETA: 11:01 - loss: 1.3885 - acc: 0.26 - ETA: 10:51 - loss: 1.3885 - acc: 0.26 - ETA: 10:41 - loss: 1.3887 - acc: 0.25 - ETA: 10:31 - loss: 1.3885 - acc: 0.25 - ETA: 10:22 - loss: 

9912/9912 [==============================] - ETA: 13:28 - loss: 1.3851 - acc: 0.26 - ETA: 13:41 - loss: 1.3862 - acc: 0.25 - ETA: 13:30 - loss: 1.3905 - acc: 0.24 - ETA: 13:21 - loss: 1.3874 - acc: 0.23 - ETA: 13:07 - loss: 1.3893 - acc: 0.24 - ETA: 12:54 - loss: 1.3912 - acc: 0.23 - ETA: 12:41 - loss: 1.3905 - acc: 0.23 - ETA: 12:29 - loss: 1.3904 - acc: 0.23 - ETA: 12:19 - loss: 1.3908 - acc: 0.23 - ETA: 12:09 - loss: 1.3907 - acc: 0.23 - ETA: 12:01 - loss: 1.3900 - acc: 0.23 - ETA: 11:52 - loss: 1.3890 - acc: 0.24 - ETA: 11:43 - loss: 1.3883 - acc: 0.25 - ETA: 11:38 - loss: 1.3883 - acc: 0.25 - ETA: 11:28 - loss: 1.3887 - acc: 0.24 - ETA: 11:17 - loss: 1.3885 - acc: 0.24 - ETA: 11:07 - loss: 1.3882 - acc: 0.25 - ETA: 10:59 - loss: 1.3879 - acc: 0.25 - ETA: 10:49 - loss: 1.3883 - acc: 0.25 - ETA: 10:40 - loss: 1.3883 - acc: 0.25 - ETA: 10:31 - loss: 1.3884 - acc: 0.25 - ETA: 10:22 - loss: 1.3880 - acc: 0.25 - ETA: 10:13 - loss: 1.3876 - acc: 0.25 - ETA: 10:04 - loss: 1.3874 - acc: 0.

In [10]:
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 1.38951599367
Test accuracy: 0.250403225806


In [13]:
Results = model.predict(x_test)

## Saving the model:

In [14]:
model.save('my_model.h5')

## Loading the model:

In [15]:
model = load_model('my_model.h5')

## Testing the model:

In [17]:
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 1.38951599367
Test accuracy: 0.250403225806


In [16]:
im = io.imread("F:\\Work\\Python Machine Learning\\CLS-Python\\CLS-Python\\Machine_learning\\BloodCells_deepLearning\\TEST\\NEUTROPHIL" + "\\" + "_0_768.jpeg", as_grey=True)
im = misc.imresize(im , (240, 240))
im = np.array(im)
im = im.astype('float32')
im /= 255
im = np.array(im).reshape(-1, 240,240,1)

#weights = model.load_weights(saved_weights_name)

Results = model.predict(im)
print(Results)
print("%.2f" % Results[0][0])
print("%.2f" % Results[0][1])
print("%.2f" % Results[0][2])
print("%.2f" % Results[0][3])

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


[[ 0.60370731  0.55057102  0.66479295  0.55031967]]
0.60
0.55
0.66
0.55
